In [ ]:
from google.colab import files

# Upload the kaggle.json file
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"armanmanukyan28","key":"75b722dd632d941c5851f53270ce89d4"}'}

In [ ]:
import os
import shutil

# Create the .kaggle folder
os.makedirs('/root/.kaggle', exist_ok=True)

# Move kaggle.json to the correct folder
shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')

# Set the required permissions for the API token
os.chmod('/root/.kaggle/kaggle.json', 600)

print("Kaggle API setup complete!")


Kaggle API setup complete!


In [ ]:
!pip install kaggle


In [ ]:
!kaggle datasets download -d jangedoo/utkface-new


Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
100% 331M/331M [00:02<00:00, 145MB/s]
100% 331M/331M [00:02<00:00, 169MB/s]


In [ ]:
import zipfile

# Extract the dataset
zip_path = 'utkface-new.zip'
extract_dir = '/content/UTKFace'

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Dataset extracted successfully!")


Dataset extracted successfully!


In [ ]:
import os

BASE_DIR = '/content/UTKFace/UTKFace'
print("Files in the dataset:", len(os.listdir(BASE_DIR)))


Files in the dataset: 23708


In [ ]:
from google.colab import files
import os
import shutil
import zipfile
import random

# Upload the kaggle.json file
files.upload()

# Create the .kaggle folder
os.makedirs('/root/.kaggle', exist_ok=True)

# Move kaggle.json to the correct folder
shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')

# Set the required permissions for the API token
os.chmod('/root/.kaggle/kaggle.json', 600)

print("Kaggle API setup complete!")

# Install Kaggle
!pip install kaggle

# Download the dataset
!kaggle datasets download -d jangedoo/utkface-new

# Extract the dataset
zip_path = 'utkface-new.zip'
extract_dir = '/content/UTKFace'

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Dataset extracted successfully!")

# Set the base directory
BASE_DIR = '/content/UTKFace/UTKFace'

# List all files in the dataset directory
all_files = os.listdir(BASE_DIR)
print("Total files in the dataset:", len(all_files))

# Shuffle and select only 100 files
random.seed(42)  # For reproducibility
selected_files = random.sample(all_files, 100)

# Create a directory for the subset
subset_dir = '/content/UTKFace_subset'
os.makedirs(subset_dir, exist_ok=True)

# Copy the selected files to the subset directory
for file_name in selected_files:
    shutil.copy(os.path.join(BASE_DIR, file_name), subset_dir)

print("Subset created with 100 files at:", subset_dir)
print("Files in the subset:", len(os.listdir(subset_dir)))


Saving kaggle.json to kaggle.json
Kaggle API setup complete!
Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
100% 330M/331M [00:11<00:00, 32.9MB/s]
100% 331M/331M [00:11<00:00, 29.2MB/s]
Dataset extracted successfully!
Total files in the dataset: 23708
Subset created with 100 files at: /content/UTKFace_subset
Files in the subset: 100


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm.notebook import tqdm
from PIL import Image

warnings.filterwarnings('ignore')
%matplotlib inline

import tensorflow as tf
from keras import initializers, layers
import keras.backend as K
from keras.preprocessing.image import load_img
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input, BatchNormalization

In [ ]:
image_paths = []
age_labels = []
gender_labels = []

for filename in tqdm(os.listdir(BASE_DIR)):
   image_path = os.path.join(BASE_DIR, filename)
   temp = filename.split('_')
age = int(temp[0])
  gender = int(temp[1])
    image_paths.append(image_path)
   age_labels.append(age)
  gender_labels.append(gender)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 11)

In [ ]:
df = pd.DataFrame()
df['image'], df['age'], df['gender'] = image_paths, age_labels, gender_labels
df.head()

In [ ]:
# @title Gender Distribution

import matplotlib.pyplot as plt

# Assuming 'gender' column contains 0 for Male and 1 for Female
gender_counts = df['gender'].value_counts()
labels = ['Male', 'Female']

plt.figure(figsize=(6, 6))
plt.pie(gender_counts, labels=labels, autopct='%1.1f%%', startangle=90)
_ = plt.title('Gender Distribution')

In [ ]:
gender_dict = {0:'Male', 1:'Female'}

In [ ]:
plt.figure(figsize=(20, 20))
files = df.iloc[10:20]

for index, file, age, gender in files.itertuples():
    plt.subplot(5, 5, index+1)
    img = load_img(file)
    img = np.array(img)
    plt.imshow(img)
    plt.title(f"Age: {age} Gender: {gender_dict[gender]}")
    plt.axis('off')

In [ ]:
def plot_data(name, data):
    plt.figure(figsize=(5, 5))
    palette_color = sns.color_palette('pastel')
    explode = [0.1 for _ in range(data.nunique())]

    target_counts = df.groupby(name)[name].count()

    target_counts.plot.pie(
    colors=palette_color,
    explode=explode,
    autopct="%1.1f%%",
    shadow=True,
    startangle=140,
    textprops={'fontsize': 14},
    wedgeprops={'edgecolor': 'black', 'linewidth': 1.5}
    )

    plt.title(name, fontsize=18, weight='bold')
    plt.axis('equal')
    plt.show()


plot_data('gender', df['gender'])

In [ ]:
sns.distplot(df['age']);

## **Feature Extraction**

In [ ]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode="grayscale")  # Load image in grayscale
        img = img.resize((128, 128), Image.Resampling.LANCZOS)  # Resize to 128x128
        img = np.array(img)  # Convert to NumPy array
        features.append(img)

    features = np.array(features)
    features = features.reshape(len(features), 128, 128, 1)  # Add channel dimension
    return features

In [ ]:
X = extract_features(df['image'])
X = X/255.0

In [ ]:
y_gender = np.array(df['gender'])
y_age = np.array(df['age'])

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Combine labels
y = np.array(list(zip(y_gender, y_age)))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Separate gender and age labels
y_gender_train, y_age_train = y_train[:, 0], y_train[:, 1]
y_gender_test, y_age_test = y_test[:, 0], y_test[:, 1]

# **Model Creation**

In [ ]:
inputs = Input((128, 128, 1))

conv_1 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv_1') (inputs)
batch_1 = BatchNormalization()(conv_1)
conv_2 = Conv2D(32, 5, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal') (batch_1)
maxp_1 = MaxPooling2D() (conv_2)
dropout_1 = Dropout(0.25) (maxp_1)

flatten = Flatten() (dropout_1)

# fully connected layers
dense_1 = Dense(128, activation = 'relu', kernel_initializer = 'he_normal') (flatten)
dense_2 = Dense(128, activation = 'relu', kernel_initializer = 'he_normal') (flatten)

batch_2 = BatchNormalization() (dense_1)
batch_3 = BatchNormalization() (dense_2)

dropout_2 = Dropout(0.5) (batch_2)
dropout_3 = Dropout(0.5) (batch_3)

output_1 = Dense(1, activation='sigmoid', name='gender_out') (dropout_2)
output_2 = Dense(1, activation='relu', name='age_out') (dropout_3)


model = Model(inputs=[inputs], outputs=[output_1, output_2])

model.compile(loss=['binary_crossentropy', 'mae'], optimizer='adam', metrics=['accuracy', 'mae'])

In [ ]:

history = model.fit(
    x=X_train,
    y=[y_gender_train, y_age_train],
    batch_size=32,
    epochs=15,
    validation_data=(X_test, [y_gender_test, y_age_test])
)


In [ ]:
# Save weights
model.save_weights('age_gender_weights.weights.h5')  # Save in HDF5 format


In [ ]:

model.load_weights('age_gender_weights.weights.h5')


In [ ]:
# Evaluate the model
evaluation_results = model.evaluate(
    x=X_test,
    y=[y_gender_test, y_age_test]
)

# Unpack the evaluation results
test_loss = evaluation_results[0]  # Overall loss
test_gender_loss = evaluation_results[1]  # Gender loss
test_age_mae = evaluation_results[2]  # Age MAE
test_gender_accuracy = evaluation_results[3]  # Gender accuracy

# Print results
print(f"Test Loss (Overall): {test_loss}")
print(f"Test Gender Loss: {test_gender_loss}")
print(f"Test Gender Accuracy: {test_gender_accuracy}")
print(f"Test Age MAE: {test_age_mae}")




In [ ]:
!apt-get install graphviz -y # Install Graphviz
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, rankdir='TB', dpi=96, expand_nested=False)
# Added to_file to save the image,dpi for resolution control,expand_nested=False to avoid large graphs

**# Plot the Results**

In [ ]:
acc = history.history['gender_out_accuracy']
val_acc = history.history['val_gender_out_accuracy']
epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Gender Accuracy Graph')
plt.legend()
plt.show()


In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Loss Graph')
plt.legend()
plt.show()


In [ ]:
age_mae = history.history['age_out_mae']
val_age_mae = history.history['val_age_out_mae']

plt.plot(epochs, age_mae, 'b', label='Training MAE')
plt.plot(epochs, val_age_mae, 'r', label='Validation MAE')
plt.title('Age MAE Graph')
plt.legend()
plt.show()


In [ ]:
gender_preds, age_preds = model.predict(X_test)
gender_preds = np.round(gender_preds).astype(int)
age_preds = age_preds.flatten()

# Calculate evaluation metrics
from sklearn.metrics import classification_report, mean_squared_error
print("Gender Classification Report:")
print(classification_report(y_gender_test, gender_preds))



# **Prediction with Test Data**

In [ ]:
image_index = 1000
print("Original Gender:", gender_dict[y_gender[image_index]], "Original Age:", y_age[image_index])

# Predict from the model
pred = model.predict(X[image_index].reshape(1, 128, 128, 1))
pred_gender = gender_dict[round(pred[0][0][0])]
pred_age = round(pred[1][0][0])

print("Predicted Gender:", pred_gender, "Predicted Age:", pred_age)
plt.axis('off')
plt.imshow(X[image_index].reshape(128, 128), cmap='gray')
plt.show()


In [ ]:
import random

# Predict random image
image_index = random.randint(0, len(X) - 1)  # Generate a random index within the dataset range
print("Original Gender:", gender_dict[y_gender[image_index]], "Original Age:", y_age[image_index])

# Predict from the model
pred = model.predict(X[image_index].reshape(1, 128, 128, 1))
pred_gender = gender_dict[round(pred[0][0][0])]
pred_age = round(pred[1][0][0])

print("Predicted Gender:", pred_gender, "Predicted Age:", pred_age)
plt.axis('off')
plt.imshow(X[image_index].reshape(128, 128), cmap='gray')
plt.show()


In [ ]:
image_index = 500 + random.randint(0, len(X) - 501)  # Generate random index starting from 500

print("Original Gender:", gender_dict[y_gender[image_index]], "Original Age:", y_age[image_index])

# Predict from the model
pred = model.predict(X[image_index].reshape(1, 128, 128, 1))
pred_gender = gender_dict[round(pred[0][0][0])]
pred_age = round(pred[1][0][0])

print("Predicted Gender:", pred_gender, "Predicted Age:", pred_age)
plt.axis('off')
plt.imshow(X[image_index].reshape(128, 128), cmap='gray')
plt.show()


# **Race Prediction**

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload the file from your local system


In [ ]:
import shutil
shutil.move('shape_predictor_68_face_landmarks.dat', '/content/shape_predictor_68_face_landmarks.dat')


In [ ]:
!pip install dlib



In [ ]:
import dlib

predictor = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')


In [ ]:
import cv2
import numpy as np
import random
import dlib
from imutils import face_utils
from matplotlib import pyplot as plt
from PIL import Image

# Load the pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

def analyze_skin_tone_and_features(image_path):
    # Read the image
    img = cv2.imread(image_path)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect face and extract landmarks
    faces = detector(gray)
    if len(faces) == 0:
        return "No face detected"

    face = faces[0]
    landmarks = predictor(gray, face)
    landmarks = face_utils.shape_to_np(landmarks)

    # Extract skin region
    left_cheek = landmarks[2:5]
    right_cheek = landmarks[13:16]
    cheeks = np.concatenate((left_cheek, right_cheek), axis=0)
    mask = np.zeros_like(gray)
    cv2.fillConvexPoly(mask, cheeks, 255)
    skin_region = cv2.bitwise_and(img_hsv, img_hsv, mask=mask)

    # Extract features
    brightness = np.mean(skin_region[:, :, 2])
    hue = np.mean(skin_region[:, :, 0])
    saturation = np.mean(skin_region[:, :, 1])

    # Debugging information
    print(f"Brightness: {brightness}, Hue: {hue}, Saturation: {saturation}")

    # Refined race prediction logic
    if brightness < 20:
        return "Black"
    elif 20 <= brightness < 40:
        if saturation > 20 and hue < 2.0:
            return "Hispanic"
        elif hue >= 2.0 and saturation <= 20:
            return "Asian"
    elif brightness >= 40:
        if hue <= 2.0 and saturation < 20:
            return "White"
        elif hue > 3.0:
            return "Asian"
    return "Multiracial or Ambiguous"


# Randomly select an image
random_index = random.randint(0, len(df) - 1)
random_image_path = df.iloc[random_index]['image']
print(f"Randomly selected image: {random_image_path}")

# Predict race
predicted_race = analyze_skin_tone_and_features(random_image_path)
print(f"Predicted Race: {predicted_race}")

# Display the image
img = Image.open(random_image_path)
plt.imshow(img)
plt.axis('off')
plt.title(f"Predicted Race: {predicted_race}")
plt.show()
